In [1]:
import os
import yaml
import numpy as np
import pandas as pd

In [8]:
with open('./package.yaml') as f:
    file = yaml.load(f, Loader=yaml.FullLoader)
    POSEDATAPATH = file["path"]["pose_keypoints"]
    FACEDATAPATH = file["path"]["faceMesh_keypoints"]
    LABELPATH = file["path"]["labelPATH"]

In [9]:
label = pd.read_excel(LABELPATH)

In [26]:
label = label[["번호", "파일명", "한국어"]].sort_values(by="번호", ascending=True).iloc[:2000]   # 컬럼 인덱싱 && 정렬 && 행 2000개 가져오기
label

,번호,파일명,한국어
10460,1,KETI_SL_0000000001.MOV,0
10440,2,KETI_SL_0000000002.MOV,1
10420,3,KETI_SL_0000000003.MOV,2
10399,4,KETI_SL_0000000004.MOV,3
10379,5,KETI_SL_0000000005.MOV,4
...,...,...,...
804,1996,KETI_SL_0000001996.MOV,트랙터
784,1997,KETI_SL_0000001997.MOV,트럭
764,1998,KETI_SL_0000001998.MOV,파도
9703,1999,KETI_SL_0000001999.MOV,38


In [27]:
pose_datapack = []
face_datapack = []

for file in os.listdir(POSEDATAPATH):
    r_path = os.path.join(POSEDATAPATH, file)
    data = np.load(r_path)
    pose_datapack.append(data)

for file in os.listdir(FACEDATAPATH):
    r_path = os.path.join(FACEDATAPATH, file)
    data = np.load(r_path)
    face_datapack.append(data)

In [29]:
label["pose"] = pose_datapack
label["face"] = face_datapack
label.to_csv('./sample_data/sample_label.csv')

In [30]:
label

,번호,파일명,한국어,pose,face
10460,1,KETI_SL_0000000001.MOV,0,"[[[0.4992966651916504, 0.1892591118812561, -0....","[[[0.5109930634498596, 0.2379748523235321, -0...."
10440,2,KETI_SL_0000000002.MOV,1,"[[[0.5037198662757874, 0.1921842098236084, -0....","[[[0.5113269090652466, 0.23961305618286133, -0..."
10420,3,KETI_SL_0000000003.MOV,2,"[[[0.5087120532989502, 0.20214205980300903, -0...","[[[0.5129885673522949, 0.2409621924161911, -0...."
10399,4,KETI_SL_0000000004.MOV,3,"[[[0.5043836236000061, 0.19213354587554932, -0...","[[[0.5124679207801819, 0.23866486549377441, -0..."
10379,5,KETI_SL_0000000005.MOV,4,"[[[0.5073361396789551, 0.19103576242923737, -0...","[[[0.5155752301216125, 0.23983445763587952, -0..."
...,...,...,...,...,...
804,1996,KETI_SL_0000001996.MOV,트랙터,"[[[0.5415878891944885, 0.19446347653865814, -0...","[[[0.5465034246444702, 0.24375079572200775, -0..."
784,1997,KETI_SL_0000001997.MOV,트럭,"[[[0.5347150564193726, 0.1984015852212906, -0....","[[[0.538702666759491, 0.24272280931472778, -0...."
764,1998,KETI_SL_0000001998.MOV,파도,"[[[0.542067289352417, 0.18870824575424194, -0....","[[[0.5389151573181152, 0.2471468150615692, -0...."
9703,1999,KETI_SL_0000001999.MOV,38,"[[[0.5323246717453003, 0.1797839254140854, -0....","[[[0.5125248432159424, 0.24749013781547546, -0..."
